# Lab 4: Using LangChain with IBM WatsonX

## 1. Intro to LangChain

[LangChain](https://docs.langchain.com/docs/) is an open-source development framework designed to simplify the creation of applications using large language models (LLMs).

The core idea of the library is that we can "chain" together different components to create more advanced use cases around LLMs. Here are the main components for the LangChain

- Model: interact with various LLMs
- Prompts: text that is sent to the LLMs
- Chains: allow to combine different LLM calls and actions automatically
- Embeddings and Vector Stores: break large data into chunks and store those to be queried when relevant
- Agents: enbale the LLMs to dynamically decide which tools to use in order to best respond to a given query

In short, **Langchain is a framework that can orchestrate a series of prompts to achieve a desired outcomes.**


## 2. How to connect LangChain to WatsonX.ai

In [23]:
import os
from typing import Any, List, Mapping, Optional, Union, Dict
from pydantic import BaseModel, Extra
try:
    from langchain import PromptTemplate
    from langchain.document_loaders import WebBaseLoader
    from langchain.chains.summarize import load_summarize_chain
    from langchain.chains import LLMChain, SimpleSequentialChain
    from langchain.chains.llm import LLMChain
    from langchain.prompts import PromptTemplate
    from langchain.chains.combine_documents.stuff import StuffDocumentsChain
    from langchain.document_loaders import PyPDFLoader
    from langchain.indexes import VectorstoreIndexCreator #vectorize db index with chromadb
    from langchain.embeddings import HuggingFaceEmbeddings #for using HugginFace embedding models
    from langchain.text_splitter import CharacterTextSplitter #text splitter
    from langchain.llms.base import LLM
    from langchain.llms.utils import enforce_stop_tokens
except ImportError:
    raise ImportError("Could not import langchain: Please install ibm-generative-ai[langchain] extension.")

from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

In [5]:
#config Watsonx.ai environment

api_key = "API_KEY"
ibm_cloud_url = "https://us-south.ml.cloud.ibm.com"
project_id = "PROJECT_ID"


creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

In [6]:
##initializing WatsonX model
params = {
    GenParams.DECODING_METHOD: "sample",
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.RANDOM_SEED: 42,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P:1
}

model = Model(
    model_id='google/flan-ul2',
    params=params,
    credentials=creds,
    project_id=project_id)


In order to use WatsonX-based LLMs with Langchain, the LLM object must be of class `BaseLanguageModel` (see [Langchain docs](https://api.python.langchain.com/en/latest/schema/langchain.schema.language_model.BaseLanguageModel.html)). We'll use the custom class below to accomplish this.

In [9]:
# Wrap the WatsonX Model in a langchain.llms.base.LLM subclass to allow LangChain to interact with the model

class LangChainInterface(LLM, BaseModel):
    credentials: Optional[Dict] = None
    model: Optional[str] = None
    params: Optional[Dict] = None
    project_id : Optional[str]=None

    class Config:
        """Configuration for this pydantic object."""
        extra = Extra.forbid

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        _params = self.params or {}
        return {
            **{"model": self.model},
            **{"params": _params},
        }
    
    @property
    def _llm_type(self) -> str:
        """Return type of llm."""
        return "IBM WATSONX"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """Call the WatsonX model"""
        params = self.params or {}
        model = Model(model_id=self.model, params=params, credentials=self.credentials, project_id=self.project_id)
        text = model.generate_text(prompt)
        if stop is not None:
            text = enforce_stop_tokens(text, stop)
        return text

llm_model = LangChainInterface(model='google/flan-ul2', credentials=creds, params=params, project_id=project_id)

In [6]:
##predict with the model
text = "Where is the capital of South Korea"
llm_model(text)

'seoul'

## 3. Prompt Templates & Chains

In the previous example, the user input is sent directly to the LLM. However, when using an LLM in an application, you will usually need to reuse the same prompt across multiple scenarios

- Accepting user input and contruct a prompt
- Generating mutiple prompts from an collection of data points in a dataset 

In [10]:
# Define the prompt templates
prompt = PromptTemplate(
  input_variables=["country"],
  template= "where is the capital of {country}?",
)

# Chaining 
chain = LLMChain(llm=llm_model, prompt=prompt)

# Getting predictions
countries = ["USA", "England", "Japan", "Saudi Arabia"]
for country in countries:
    response = chain.run(country)
    print(prompt.format(country=country) + " = " + response)

where is the capital of USA? = washington dc
where is the capital of England? = london
where is the capital of Japan? = tokyo
where is the capital of Saudi Arabia? = jeddah


## 4. Simple sequential chains
The utility of LangChain becomes apparent as we chain outputs of one model as input to another model. Here's a simple example where one generates a question which the other model answers.

LangChain determines a model's output based on its response.  In our examples, the first model creates a response to the end prompt of "Question:" which LangChain maps as an input variable called "question" which it passes to the 2nd model.

In [11]:
## Create two sequential prompts 
pt1 = PromptTemplate(input_variables=["topic"], template="Generate a random question about {topic}: Question: ")
pt2 = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)

In [12]:
flan = LangChainInterface(model='google/flan-ul2', credentials=creds, params=params, project_id=project_id)
model = LangChainInterface(model='google/flan-ul2', credentials=creds, project_id=project_id)

In [13]:
prompt_to_flan = LLMChain(llm=flan, prompt=pt1)
flan_to_model = LLMChain(llm=model, prompt=pt2)
qa = SimpleSequentialChain(chains=[prompt_to_flan, flan_to_model], verbose=True)

In [16]:
qa.run("artificial intelligence")



> Entering new SimpleSequentialChain chain...
What does the term "artificial intelligence" mean?
intelligent computer program

> Finished chain.


'intelligent computer program'

## 5. Summarization

In [45]:
# Initialize llm and document loader:
print("Loading web document...")
# Try out some other documents as well
loader = WebBaseLoader("https://www.ibm.com/blog/reducing-defects-and-downtime-with-ai-enabled-automated-inspections/")
doc = loader.load()
print("Done.")

# You might need to tweak some of the runtime parameters to optimize the results.
print("Initializing mixtral-8x7b model...")
params = {
    GenParams.DECODING_METHOD: "sample",
    GenParams.TEMPERATURE: 0.15,
    GenParams.TOP_P: 1,
    GenParams.TOP_K: 20,
    GenParams.REPETITION_PENALTY: 1.0,
    GenParams.MIN_NEW_TOKENS: 20,
    GenParams.MAX_NEW_TOKENS: 205,
    GenParams.STOP_SEQUENCES: ["\n"]
}

mixtral_model = Model(
    model_id="ibm-mistralai/mixtral-8x7b-instruct-v01-q",
    params=params,
    credentials=creds,
    project_id=project_id
).to_langchain()

# Define prompt
prompt_template = """Write a concise summary of the following article in one paragraph:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
print("Initializing chain...")
llm_chain = LLMChain(llm=mixtral_model, prompt=prompt)

# Define StuffDocumentsChain
print("Stuff chain with documents...")
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="text"
)

print("Running summarization on stuffed document chain...\n")
res = stuff_chain.run(doc)

print(res)

print("\nDone.")

Loading web document...
Done.
Initializing mixtral-8x7b model...
Initializing chain...
Stuff chain with documents...
Running summarization on stuffed document chain...



IBM helped a multinational automobile manufacturer reduce defects and downtime in their manufacturing process by deploying AI-enabled automated inspections. The solutions included fixed-mounted, handheld, and wearable inspections, which were based on standard iPhones and used readily available hardware. The lightweight and portable nature of IBM's solution, combined with its ability to be used anywhere at any time, was a major selling point for the client. The IBM Inspection Suite improved the client's quality inspection process without requiring coding and was simple to train and deploy. The system learned quickly from images of acceptable and defective work products, enabling it to be up and running within a matter of weeks. The implementation costs were also lower than those of viable alternatives. The ability to d